##### Объектно-Ориентированное Программирование (ООП)

In [39]:
'''
    Ссылка на документацию: https://docs.python.org/3/tutorial/classes.html
    Обычно в контексте ООП говорят о классах, наследовании, инкапсуляции и полиморфизме.
    Так что основной объект, с которым необходимо уметь работать -- это класс.

    Первый принцип ООП, инкапсуляция, - это принцип объектно-ориентированного программирования,
        который заключается в скрытии деталей реализации от внешнего мира и предоставлении интерфейса
        для взаимодействия с объектом.

        @ Нейро (яндекс)

'''

import datetime
from abc import ABC, abstractmethod
from typing import override

# создаем абстрактный класс опциона
'''

'''
class Option(ABC):
    # самый первый dunder метод, который все используют и иногда не знают, что это он и есть
    def __init__(
        self,
        price: float,
        strike: float,
        expiration_date: str,
        name: str = 'Abstract Option'
    ) -> None:
        super().__init__()
        assert strike > 0, 'Strike must be positive! Got {strike}'
        self.name = name
        self.price = price
        self.strike = strike
        self.expiration_date = expiration_date

    # этот dunder метод позволяет представлять объект класса в удобочитаемой форме
    def __repr__(self) -> str:
        return f'{self.name}. Strike = {self.strike}. Expiration date = {self.expiration_date}'

    # используем декоратор, который не позволит инстанциировать асбтрактный класс
    # пример инкапсуляции -- мы описываем, как рассчитывается выплата по опциону,
    # но для внешнего пользователя логика этой операции скрыта и он только использует предоставляемый ему метод get_payoff
    @abstractmethod
    def get_payoff(self, expiration_price: float) -> float:
        # этот метод должен быть имплементирован в дочерних классах
        pass

'''
    Второй принцип, наследование, - это один из ключевых принципов объектно-ориентированного программирования,
        который позволяет создавать новые классы на основе уже существующих.

        @ Нейро (яндекс)
'''

class EuropeanCallOption(Option): # наследуем все самое лучшее от базового класса
    def __init__(
        self,
        *args,
        name='EuropeanCallOption',
        **kwargs
    ) -> None:
        super().__init__(*args, name=name, **kwargs) # так что нам не нужно заново описывать поля и метод инстанциирования экземпляра

    # но нужно переписать (override) абстрактный метод
    # декоратор override в явном виде указывает, что этот метод переопределяет родительский
    @override
    def get_payoff(self, expiration_price: float) -> float:
        return max(expiration_price - self.strike, 0)

In [40]:
# a = Option(10, 100, '11') # ошибка TypeError
b = EuropeanCallOption(10, 100, '11') # ok
b.get_payoff(111)

11

In [41]:
'''
    NAMESPACES
    Пространство имен (namespace) -- это словарь, которые отображает имена переменных в объекты.
    Пространства имен в python делятся на 3 группы по уровню вложенности:
    1. Глобальное пространство имен
    2. Промежуточное пространство имен
    3. Локальное пространство имен

    Глобальное пространство имен отображает объекты текущего модуля и всех импортированных модулей и подмодулей.
    Промежеточное пространство имен отображает объекты внутри какой-то области (функции или класса),
        которая является более глобальной по отношению к структуре, вложенной в текущую.
    Локальное пространство имен -- это отображение для самой вложенной структуры

    Структура пространств имен таким образом имеет вид:

    global_namespace = {                                # глобальное пространство имен, включает структуры, которые имеют вложенность (т.е. другие пр-ва имен)
        'global_object_name': global_object,
        ...
        'nonlocal_namespace': {                         # пространство имен следующего уровня вложенности, промежуточное пространство имен
            'nonlocal_object_name': nonlocal_object,
            ...
            'local_namespace': {                        # локальное пространство имен, самая вложенная структура
                'local_object_name': local_object,
                ...
            }
        }
    }

    При обсуждении классов мы можем рассматривать их как некоторые структуры-контейнеры, которые имеют собственные пространства имен.
    Чтобы получить доступ к пространству имен (к словарю), нужно вызвать свойство __dict__ модуля (класса)
    Например:
'''
global_namespace = globals()
eu_namespace = global_namespace['b'].__dict__
print(eu_namespace)

{'name': 'EuropeanCallOption', 'price': 10, 'strike': 100, 'expiration_date': '11'}


In [42]:
from abc import ABC, abstractmethod
from typing import Sequence, Iterator, Generator

# опишем абстрактный актив, который можно включить в портфель
# (мы могли бы его использовать как базовый класс для класса Option)
class AbstractAsset(ABC):
    def __init__(
        self,
        price: float,
        name: str = 'AbstractAsset'
    ) -> None:
        self.price = price
        self.name = name

    def __repr__(self) -> str:
        return f'{self.name}. Price = {self.price}'

# dummy класс, который нужен только виртуально
#  и будет переопределен
class Portfolio:
    pass

class Portfolio(ABC):
    PORTFOLIOS: int = 0  # счетчик, который будет отслеживать новые экземпляры

    def __init__(
        self,
        assets: Sequence[AbstractAsset] = list(),
        name: str = 'Portfolio'
    ) -> None:
        self.name = name
        self.assets = assets
        self.__price = self.price

    # property -- свойство -- специальный декоратор, который дает возможность манипулировать аттрибутами класса до их возвращения или изменения
    # в частности, как ниже, мы сначала рассчитываем стоимость портфеля как сумму его активов, прежде чем вернуть значение,
    # так что каждый раз мы получаем обновленное актуальное значение по стоимости
    @property
    def price(self):
        price = sum(asset.price for asset in self.assets) if len(self.assets) > 0 else 0
        self.__price = price
        return self.__price

    # позволяет вычислять "длину" экземпляра класса
    def __len__(self) -> int:
        return len(self.assets)

    # позволяет сделать из экземпляра итератор, для которого можно применить next(obj)
    def __iter__(self) -> Generator:
        return (asset for asset in self.assets)

    # функция позволяет складывать объекты Portfolio
    # и возвращает новый экземпляр класса
    def __add__(self, other: Portfolio|AbstractAsset) -> Portfolio: # <- здесь был нужен определенный выше dummy class
        if isinstance(other, Portfolio):
            new_assets = self.assets + other.assets
        elif isinstance(other, AbstractAsset):
            new_assets = self.assets + [AbstractAsset]
        else:
            # если мы пытаемся сложить портфель с грушами, вероятно мы делаем что-то не то,
            # надо об этом сообщить
            raise TypeError('Нельзя складывать портфель с чем-то непонятным!')
        # простоты ради (и эффективности!)
        # мы можем создать экземпляр класса, распаковав словарь с помощью **
        # и передав в конструктор все необходимые параметры, взятые из экземпляра донора,
        # а не переписывать все поля вручную
        kwargs = {
            key: value for key, value in self.__dict__.items()
            if not key.startswith('_')
        }
        kwargs.update(
            {
                'assets': new_assets
            }
        )
        return Portfolio(
            **kwargs
        )



In [43]:
# ПРАКТИЧЕСКОЕ ЗАДАНИЕ 1: EuropeanPutOption
# Реализуйте класс EuropeanPutOption, наследующий от Option
# Payoff для put опциона: max(strike - expiration_price, 0)

class EuropeanPutOption(Option):
    def __init__(
        self,
        *args,
        name='EuropeanPutOption',
        **kwargs
    ) -> None:
        # TODO: Реализуйте конструктор
        pass

    @override
    def get_payoff(self, expiration_price: float) -> float:
        # TODO: Реализуйте расчет payoff для put опциона
        pass

# ПРАКТИЧЕСКОЕ ЗАДАНИЕ 2: Stock класс
# Создайте класс Stock как наследник AbstractAsset

class Stock(AbstractAsset):
    def __init__(
        self,
        price: float,
        symbol: str,
        name: str = 'Stock'
    ) -> None:
        # TODO: Реализуйте конструктор
        pass

    def __repr__(self) -> str:
        # TODO: Реализуйте красивый вывод
        pass

# ПРАКТИЧЕСКОЕ ЗАДАНИЕ 3: Portfolio.__repr__
# Реализуйте метод __repr__ для класса Portfolio

# TODO: Добавьте метод __repr__ в класс Portfolio
# Пример вывода: "Portfolio 'My Portfolio' (3 assets, total value: $150.00)"


In [44]:
'''
    Задачи: имплементировать в виде классов производные финансовые инструменты
'''

'\n    Задачи: имплементировать в виде классов производные финансовые инструменты\n'

##### DUNDER МЕТОДЫ

In [45]:
'''
    Семантически dunder -- сокращение от dobule underscore (двойное подчеркивание).
    Синтаксически dunder -- это специальный метод из списка таких методов, который отличается двойным подчеркивание до и после своего имени.
    То есть __init__, __repr__, представленные выше это dunder методы.
    Эти методы позволяют объекту класса иметь специальные возможности.
    Например, __repr__ позволяет выводить экземпляр класса в удобочитаемой форме.
    Другие, например, __iter__ позволяет создавать итератор из экземпляра класса (iterator_object = iter(instance_of_my_class_with_iter_))
    Очень хорошая статья про dunder методы -- https://habr.com/ru/companies/timeweb/articles/876048/
'''

'\n    Семантически dunder -- сокращение от dobule underscore (двойное подчеркивание).\n    Синтаксически dunder -- это специальный метод из списка таких методов, который отличается двойным подчеркивание до и после своего имени.\n    То есть __init__, __repr__, представленные выше это dunder методы. \n    Эти методы позволяют объекту класса иметь специальные возможности.\n    Например, __repr__ позволяет выводить экземпляр класса в удобочитаемой форме.\n    Другие, например, __iter__ позволяет создавать итератор из экземпляра класса (iterator_object = iter(instance_of_my_class_with_iter_))\n    Очень хорошая статья про dunder методы -- https://habr.com/ru/companies/timeweb/articles/876048/\n'

In [46]:
# пример со сложением деривативов (__add__). объект Portfolio, который можно складывать между экземплярами и отдельными инструментами,
# со свойствами длины, получением по ключу, итерированием и т.д.
# Задача: доделать что-то с ним

In [47]:
pf_1 = Portfolio([AbstractAsset(price=10), AbstractAsset(price=4)])
pf_2 = Portfolio([AbstractAsset(price=12), AbstractAsset(price=-6)])
pf_3 = pf_1 + pf_2

In [48]:
assets_generator = iter(pf_3)
print(next(assets_generator))
print(len(pf_3))


AbstractAsset. Price = 10
4


In [49]:
'''
    Задача:
    написать метод __repr__,
    написать производный от Portfolio, класс AlphaPortfolio, содержаший объект Alpha
    (ее тоже имплементировать, пока как абстрактную)
'''

'\n    Задача:\n    написать метод __repr__,\n    написать производный от Portfolio, класс AlphaPortfolio, содержаший объект Alpha\n    (ее тоже имплементировать, пока как абстрактную)\n'

##### ЗАДАЧИ ДЛЯ ПРАКТИКИ ООП

**1. БАЗОВЫЕ ФИНАНСОВЫЕ ИНСТРУМЕНТЫ:**
- Реализуйте класс `EuropeanPutOption`, наследующий от `Option`
- Реализуйте класс `AmericanCallOption` с возможностью досрочного исполнения
- Добавьте валидацию даты истечения в базовый класс `Option`
- Создайте класс `Stock` (акция) как наследник `AbstractAsset`

**2. РАСШИРЕННЫЕ ОПЦИОНЫ:**
- Реализуйте класс `BinaryOption` (бинарный опцион)
- Создайте класс `BarrierOption` (барьерный опцион)
- Добавьте метод расчета Greeks (дельта, гамма, тета, вега) для опционов

**3. ПОРТФЕЛЬ И УПРАВЛЕНИЕ АКТИВАМИ:**
- Реализуйте метод `__repr__` для класса `Portfolio`
- Добавьте метод `__getitem__` для доступа к активам по индексу
- Реализуйте метод `__contains__` для проверки наличия актива в портфеле
- Добавьте метод `remove_asset` для удаления актива из портфеля
- Создайте метод `calculate_portfolio_risk` для расчета риска портфеля

**4. АЛЬФА-СТРАТЕГИИ:**
- Реализуйте абстрактный класс `Alpha`
- Создайте класс `AlphaPortfolio`, наследующий от `Portfolio`
- Добавьте методы для расчета альфа-коэффициента и бета-коэффициента
- Реализуйте класс `MomentumAlpha` и `MeanReversionAlpha`

**5. ДОПОЛНИТЕЛЬНЫЕ DUNDER МЕТОДЫ:**
- Добавьте `__eq__`, `__lt__`, `__le__` для сравнения портфелей
- Реализуйте `__mul__` для умножения портфеля на число (масштабирование)
- Добавьте `__str__` для красивого вывода портфеля
- Реализуйте `__bool__` для проверки, является ли портфель пустым

**6. ПРОДВИНУТЫЕ ЗАДАЧИ:**
- Создайте декоратор `@validate_option_params` для валидации параметров опционов
- Реализуйте класс `PortfolioManager` с методами для управления несколькими портфелями
- Добавьте поддержку сериализации/десериализации портфелей (pickle)
- Создайте контекстный менеджер для временного изменения параметров портфеля


In [50]:
# 1. БАЗОВЫЕ ФИНАНСОВЫЕ ИНСТРУМЕНТЫ

class EuropeanPutOption(Option):
    def __init__(self, *args, name='EuropeanPutOption', **kwargs):
        super().__init__(*args, name=name, **kwargs)

    def get_payoff(self, expiration_price: float) -> float:
        return max(self.strike - expiration_price, 0)

class AmericanCallOption(Option):
    def __init__(self, *args, name='AmericanCallOption', **kwargs):
        super().__init__(*args, name=name, **kwargs)

    def get_payoff(self, expiration_price: float) -> float:
        return max(expiration_price - self.strike, 0)

    def can_exercise_early(self, current_price: float) -> bool:
        return current_price > self.strike

class Stock(AbstractAsset):
    def __init__(self, price: float, symbol: str, name: str = 'Stock'):
        super().__init__(price, name)
        self.symbol = symbol

    def __repr__(self) -> str:
        return f"Stock('{self.symbol}', ${self.price:.2f})"

class Option:
    def __init__(self, strike: float, expiration_date: str, name: str = 'Option'):
        self.strike = strike
        self.name = name
        self._validate_expiration_date(expiration_date)
        self.expiration_date = expiration_date

    def _validate_expiration_date(self, date: str):
        # простая валидация формата даты
        if len(date) != 10 or date[4] != '-' or date[7] != '-':
            raise ValueError("Дата должна быть в формате YYYY-MM-DD")

In [51]:
# 2. РАСШИРЕННЫЕ ОПЦИОНЫ

class BinaryOption(Option):
    def __init__(self, strike: float, expiration_date: str, payout: float = 100):
        super().__init__(strike, expiration_date, 'BinaryOption')
        self.payout = payout

    def get_payoff(self, expiration_price: float) -> float:
        if expiration_price >= self.strike:
            return self.payout
        return 0

class BarrierOption(Option):
    def __init__(self, strike: float, expiration_date: str, barrier: float, option_type: str = 'up-and-out'):
        super().__init__(strike, expiration_date, 'BarrierOption')
        self.barrier = barrier
        self.option_type = option_type

    def get_payoff(self, expiration_price: float, max_price: float, min_price: float) -> float:
        if self.option_type == 'up-and-out':
            if max_price >= self.barrier:
                return 0
            return max(expiration_price - self.strike, 0)
        return 0

class OptionWithGreeks(Option):
    def calculate_delta(self, spot_price: float, time_to_expiry: float) -> float:
        # Упрощенный расчет дельты
        return (spot_price - self.strike) / (self.strike * 10)

    def calculate_gamma(self, spot_price: float) -> float:
        return 0.01

    def calculate_theta(self, time_to_expiry: float) -> float:
        return -0.05

    def calculate_vega(self, volatility: float) -> float:
        return volatility * 0.1

In [52]:
# 3. ПОРТФЕЛЬ И УПРАВЛЕНИЕ АКТИВАМИ

class Portfolio:
    def __init__(self, name: str = "My Portfolio"):
        self.name = name
        self.assets = []

    def __repr__(self) -> str:
        total_value = sum(asset.price for asset in self.assets)
        return f"Portfolio '{self.name}' ({len(self.assets)} assets, total value: ${total_value:.2f})"

    def __getitem__(self, index):
        return self.assets[index]

    def __contains__(self, asset):
        return asset in self.assets

    def __len__(self):
        return len(self.assets)

    def add_asset(self, asset):
        self.assets.append(asset)

    def remove_asset(self, asset):
        if asset in self.assets:
            self.assets.remove(asset)
            return True
        return False

    def calculate_portfolio_risk(self) -> float:
        if not self.assets:
            return 0.0

        total_value = sum(asset.price for asset in self.assets)
        if total_value == 0:
            return 0.0

        risk = 0.0
        for asset in self.assets:
            weight = asset.price / total_value
            # предполагаем, что более дорогие активы более волатильны
            asset_risk = min(asset.price * 0.1, 0.3)  # макс 30% риска
            risk += weight * asset_risk

        return risk

In [53]:
# 4. АЛЬФА-СТРАТЕГИИ

from abc import ABC, abstractmethod

class Alpha(ABC):
    @abstractmethod
    def calculate_alpha(self, portfolio) -> float:
        pass

class AlphaPortfolio(Portfolio):
    def calculate_alpha_coefficient(self, market_return: float) -> float:
        # расчет альфа-коэффициента
        portfolio_return = self.calculate_portfolio_return()
        beta = self.calculate_beta_coefficient()
        return portfolio_return - (beta * market_return)

    def calculate_beta_coefficient(self) -> float:
        # расчет бета-коэффициента
        if not self.assets:
            return 1.0
        # Предполагаем, что портфель следует за рынком
        return 1.0

    def calculate_portfolio_return(self) -> float:
        # расчет доходности
        if not self.assets:
            return 0.0
        total_value = sum(asset.price for asset in self.assets)
        return total_value * 0.05  # 5% доходность

class MomentumAlpha(Alpha):
    def calculate_alpha(self, portfolio) -> float:
        # стратегия моментума
        if len(portfolio) < 2:
            return 0.0
        # больше активов - выше альфа
        return len(portfolio) * 0.01

class MeanReversionAlpha(Alpha):
    def calculate_alpha(self, portfolio) -> float:
        # стратегия возврата к среднему
        if not portfolio.assets:
            return 0.0
        # выше риск - выше потенциальная альфа
        risk = portfolio.calculate_portfolio_risk()
        return risk * 0.5

In [54]:
# 5. ДОПОЛНИТЕЛЬНЫЕ DUNDER МЕТОДЫ

class EnhancedPortfolio(Portfolio):
    def __eq__(self, other):
        if not isinstance(other, Portfolio):
            return False
        return (self.name == other.name and
                len(self.assets) == len(other.assets))

    def __lt__(self, other):
        if not isinstance(other, Portfolio):
            return NotImplemented
        return (sum(a.price for a in self.assets) <
                sum(a.price for a in other.assets))

    def __le__(self, other):
        return self.__lt__(other) or self.__eq__(other)

    def __mul__(self, factor):
        # новый портфель с масштабированными ценами
        new_portfolio = EnhancedPortfolio(f"{self.name} (scaled)")
        for asset in self.assets:
            # новый актив с измененной ценой
            scaled_asset = type(asset)(asset.price * factor, getattr(asset, 'symbol', 'SCALED'))
            new_portfolio.add_asset(scaled_asset)
        return new_portfolio

    def __rmul__(self, factor):
        return self.__mul__(factor)

    def __str__(self):
        total_value = sum(asset.price for asset in self.assets)
        assets_list = "\n".join([f"  - {asset}" for asset in self.assets])
        return f"Портфель: {self.name}\nАктивы ({len(self.assets)}):\n{assets_list}\nОбщая стоимость: ${total_value:.2f}"

    def __bool__(self):
        return len(self.assets) > 0

In [55]:
# 6. ПРОДВИНУТЫЕ ЗАДАЧИ

from functools import wraps
import pickle

# декоратор для валидации параметров опционов
def validate_option_params(func):
    @wraps(func)
    def wrapper(strike, expiration_date, *args, **kwargs):
        if strike <= 0:
            raise ValueError("Цена страйк должна быть положительной")
        if len(expiration_date) != 10 or expiration_date[4] != '-' or expiration_date[7] != '-':
            raise ValueError("Дата должна быть в формате YYYY-MM-DD")
        return func(strike, expiration_date, *args, **kwargs)
    return wrapper

class PortfolioManager:
    def __init__(self):
        self.portfolios = {}

    def add_portfolio(self, name: str, portfolio: Portfolio):
        self.portfolios[name] = portfolio

    def get_portfolio(self, name: str) -> Portfolio:
        return self.portfolios.get(name)

    def remove_portfolio(self, name: str):
        if name in self.portfolios:
            del self.portfolios[name]

    def get_total_value(self) -> float:
        return sum(sum(asset.price for asset in portfolio.assets)
                  for portfolio in self.portfolios.values())

    def save_portfolio(self, name: str, filename: str):
        if name in self.portfolios:
            with open(filename, 'wb') as f:
                pickle.dump(self.portfolios[name], f)

    def load_portfolio(self, filename: str, name: str):
        with open(filename, 'rb') as f:
            portfolio = pickle.load(f)
            self.portfolios[name] = portfolio

# контекстный менеджер для временного изменения портфеля
class TemporaryPortfolioChange:
    def __init__(self, portfolio, temp_assets):
        self.portfolio = portfolio
        self.temp_assets = temp_assets
        self.original_assets = None

    def __enter__(self):
        self.original_assets = self.portfolio.assets.copy()
        self.portfolio.assets = self.temp_assets
        return self.portfolio

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.portfolio.assets = self.original_assets

# тестовые активы
apple = Stock(150.0, "AAPL")
google = Stock(2800.0, "GOOGL")

portfolio = EnhancedPortfolio("Тестовый портфель")
portfolio.add_asset(apple)
portfolio.add_asset(google)

print(portfolio)
print(f"Риск портфеля: {portfolio.calculate_portfolio_risk():.2%}")

doubled_portfolio = portfolio * 2
print(f"\nУдвоенный портфель: {doubled_portfolio}")

Портфель: Тестовый портфель
Активы (2):
  - Stock('AAPL', $150.00)
  - Stock('GOOGL', $2800.00)
Общая стоимость: $2950.00
Риск портфеля: 30.00%

Удвоенный портфель: Портфель: Тестовый портфель (scaled)
Активы (2):
  - Stock('AAPL', $300.00)
  - Stock('GOOGL', $5600.00)
Общая стоимость: $5900.00


In [56]:
# ПРАКТИЧЕСКОЕ ЗАДАНИЕ 4: Дополнительные dunder методы для Portfolio
# Реализуйте методы для улучшения функциональности Portfolio

# TODO: Добавьте в класс Portfolio следующие методы:

# 1. __getitem__ - доступ к активам по индексу
# portfolio[0] должен возвращать первый актив
# portfolio[1:3] должен возвращать срез активов

# 2. __contains__ - проверка наличия актива
# asset in portfolio должно возвращать True/False

# 3. __eq__ - сравнение портфелей
# portfolio1 == portfolio2 должно сравнивать по стоимости

# 4. __mul__ - умножение на число (масштабирование)
# portfolio * 2 должно создавать новый портфель с удвоенными ценами

# 5. __str__ - красивый вывод для пользователя
# str(portfolio) должен показывать детальную информацию

# 6. __bool__ - проверка на пустоту
# bool(portfolio) должно возвращать False для пустого портфеля

# Пример использования:
# pf = Portfolio([Stock(100, 'AAPL'), Stock(200, 'GOOGL')])
# print(pf[0])           # Stock('AAPL', Price = 100)
# print(Stock(100, 'AAPL') in pf)  # True
# print(pf == pf * 1)    # True
# print(pf * 2)          # Новый портфель с удвоенными ценами
# print(str(pf))         # Красивый вывод
# print(bool(pf))        # True (не пустой)


In [57]:
# ПРАКТИЧЕСКОЕ ЗАДАНИЕ 5: Alpha стратегии
# Реализуйте систему альфа-стратегий для портфеля

from abc import ABC, abstractmethod
from typing import List, Dict, Any
import numpy as np

# TODO: Реализуйте абстрактный класс Alpha
class Alpha(ABC):
    def __init__(self, name: str = 'Alpha'):
        # TODO: Реализуйте конструктор
        pass

    @abstractmethod
    def calculate_alpha(self, returns: List[float], market_returns: List[float]) -> float:
        # TODO: Реализуйте абстрактный метод расчета альфы
        pass

    def __repr__(self) -> str:
        # TODO: Реализуйте вывод
        pass

# TODO: Реализуйте класс MomentumAlpha
class MomentumAlpha(Alpha):
    def __init__(self, lookback_period: int = 20, name: str = 'MomentumAlpha'):
        # TODO: Реализуйте конструктор
        pass

    def calculate_alpha(self, returns: List[float], market_returns: List[float]) -> float:
        # TODO: Реализуйте расчет альфы на основе моментума
        # Альфа = средняя доходность за период - бета * средняя рыночная доходность
        pass

# TODO: Реализуйте класс MeanReversionAlpha
class MeanReversionAlpha(Alpha):
    def __init__(self, reversion_threshold: float = 0.02, name: str = 'MeanReversionAlpha'):
        # TODO: Реализуйте конструктор
        pass

    def calculate_alpha(self, returns: List[float], market_returns: List[float]) -> float:
        # TODO: Реализуйте расчет альфы на основе возврата к среднему
        pass

# TODO: Реализуйте класс AlphaPortfolio
class AlphaPortfolio(Portfolio):
    def __init__(
        self,
        assets: List[AbstractAsset] = None,
        alpha_strategy: Alpha = None,
        name: str = 'AlphaPortfolio'
    ):
        # TODO: Реализуйте конструктор
        pass

    def calculate_portfolio_alpha(self, market_returns: List[float]) -> float:
        # TODO: Реализуйте расчет альфы портфеля
        pass

    def calculate_beta(self, market_returns: List[float]) -> float:
        # TODO: Реализуйте расчет беты портфеля
        pass

    def get_risk_metrics(self, market_returns: List[float]) -> Dict[str, float]:
        # TODO: Реализуйте расчет различных метрик риска
        # Возвращайте словарь с ключами: 'alpha', 'beta', 'sharpe_ratio', 'volatility'
        pass


In [58]:
# ПРАКТИЧЕСКОЕ ЗАДАНИЕ 6: Продвинутые техники ООП

# TODO: Реализуйте декоратор для валидации параметров опционов
def validate_option_params(func):
    """
    Декоратор для валидации параметров опционов.
    Проверяет, что price > 0, strike > 0, expiration_date не в прошлом
    """
    def wrapper(self, price, strike, expiration_date, *args, **kwargs):
        # TODO: Добавьте валидацию
        # - price > 0
        # - strike > 0
        # - expiration_date не в прошлом (используйте datetime)
        pass
    return wrapper

# TODO: Реализуйте класс PortfolioManager
class PortfolioManager:
    """
    Менеджер для управления несколькими портфелями
    """
    def __init__(self, name: str = 'PortfolioManager'):
        # TODO: Реализуйте конструктор
        # Должен содержать словарь портфелей и методы для управления
        pass

    def add_portfolio(self, portfolio: Portfolio, name: str = None) -> None:
        # TODO: Добавьте портфель в менеджер
        pass

    def remove_portfolio(self, name: str) -> None:
        # TODO: Удалите портфель из менеджера
        pass

    def get_portfolio(self, name: str) -> Portfolio:
        # TODO: Получите портфель по имени
        pass

    def get_total_value(self) -> float:
        # TODO: Рассчитайте общую стоимость всех портфелей
        pass

    def rebalance_portfolios(self, target_weights: Dict[str, float]) -> None:
        # TODO: Ребалансируйте портфели согласно целевым весам
        pass

    def __repr__(self) -> str:
        # TODO: Реализуйте красивый вывод менеджера
        pass

# TODO: Реализуйте контекстный менеджер для временного изменения параметров портфеля
class PortfolioContext:
    """
    Контекстный менеджер для временного изменения параметров портфеля
    """
    def __init__(self, portfolio: Portfolio, **kwargs):
        # TODO: Реализуйте конструктор
        # kwargs должны содержать параметры для временного изменения
        pass

    def __enter__(self):
        # TODO: Сохраните текущие параметры и примените новые
        pass

    def __exit__(self, exc_type, exc_val, exc_tb):
        # TODO: Восстановите исходные параметры
        pass

# Пример использования контекстного менеджера:
# with PortfolioContext(portfolio, name='Temporary Portfolio') as temp_portfolio:
#     print(temp_portfolio.name)  # 'Temporary Portfolio'
# print(portfolio.name)  # Исходное имя восстановлено


In [59]:
# ПРИМЕРЫ ТЕСТИРОВАНИЯ И ПОДСКАЗКИ

# Тестирование EuropeanPutOption:
# put_option = EuropeanPutOption(price=5, strike=100, expiration_date='2024-12-31')
# print(put_option.get_payoff(90))  # Должно вернуть 10 (100-90)
# print(put_option.get_payoff(110)) # Должно вернуть 0

# Тестирование Stock:
# apple_stock = Stock(price=150, symbol='AAPL', name='Apple Inc.')
# print(apple_stock)  # Должно показать: Stock('AAPL', Price = 150)

# Тестирование Portfolio с новыми методами:
# pf = Portfolio([Stock(100, 'AAPL'), Stock(200, 'GOOGL')], name='Tech Portfolio')
# print(pf[0])                    # Первый актив
# print(Stock(100, 'AAPL') in pf) # True
# print(pf == pf * 1)             # True
# print(pf * 2)                   # Новый портфель с удвоенными ценами
# print(str(pf))                  # Красивый вывод
# print(bool(pf))                 # True

# Тестирование Alpha стратегий:
# momentum_alpha = MomentumAlpha(lookback_period=20)
# returns = [0.01, 0.02, -0.01, 0.03, 0.01]
# market_returns = [0.005, 0.015, -0.005, 0.025, 0.005]
# alpha_value = momentum_alpha.calculate_alpha(returns, market_returns)

# Тестирование AlphaPortfolio:
# alpha_pf = AlphaPortfolio(
#     assets=[Stock(100, 'AAPL'), Stock(200, 'GOOGL')],
#     alpha_strategy=momentum_alpha,
#     name='Alpha Tech Portfolio'
# )
# risk_metrics = alpha_pf.get_risk_metrics(market_returns)

# Тестирование PortfolioManager:
# manager = PortfolioManager('My Manager')
# manager.add_portfolio(pf, 'tech_portfolio')
# manager.add_portfolio(alpha_pf, 'alpha_portfolio')
# total_value = manager.get_total_value()

# ПОДСКАЗКИ ДЛЯ РЕАЛИЗАЦИИ:

# 1. Для __getitem__ используйте return self.assets[index]
# 2. Для __contains__ используйте return item in self.assets
# 3. Для __eq__ сравнивайте self.price с other.price
# 4. Для __mul__ создавайте новые активы с измененными ценами
# 5. Для __str__ используйте f-строки для красивого форматирования
# 6. Для __bool__ используйте return len(self.assets) > 0
# 7. Для Alpha расчетов используйте numpy для статистических вычислений
# 8. Для контекстного менеджера сохраняйте исходные значения в __enter__ и восстанавливайте в __exit__
